In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D
from tensorflow.keras.layers import Input, LeakyReLU, Softmax, Reshape 
from tensorflow.keras.layers import concatenate, maximum, Lambda, Layer
import pickle

### Additional layers not available in core Keras

In [30]:
def feature_pool_max(input, pool_size=2, axis=1):
    """
    Based on lasagne implementation of FeaturePool
    """
    input_shape = input.shape.as_list()
    num_feature_maps = input_shape[axis]
    num_feature_maps_out = num_feature_maps // pool_size
    
    pool_shape = tf.TensorShape(
        (input_shape[1:axis] + [num_feature_maps_out, pool_size] + input_shape[axis+1:])
    )
    
    print(pool_shape)
    input_reshaped = Reshape(pool_shape)(input)
    # reduce along all axis but the target one
    reduction_axis = list(range(1, len(pool_shape)+1))
    reduction_axis.pop(axis-1)
    
    return tf.reduce_max(input_reshaped, axis=reduction_axis)

# create Layer for reshape with batchsize
# strong_reshape_func = lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2))
# StrongReshape = Lambda(strong_reshape_func)

In [31]:
def build_model(width=512, height=512, filename=None,
                n_classes=5, batch_size=64, p_conv=0.0):
    """
    
    """
    # Input shape (height, width, depth), different from original implimentation
    main_input = Input(shape=(height, width, 3),
                       batch_size=batch_size,
                      )
    
    # Note: for conv layers paper uses untie_biases=True
    # layer will have separate bias parameters for each position in each channel. 
    # As a result, the b attribute will be a 3D tensor.

    # TODO: set names of layers, check init
    # no need to init weights as they will be loaded from a file
    # Conv layers(filters, kernel_size)
    conv_1 = Conv2D(32, 7, strides=(2, 2), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(main_input)
    dropout_1 = Dropout(p_conv)(conv_1)
    maxpool_1 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_1)
    # 3
    conv_2 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(maxpool_1)
    dropout_2 = Dropout(p_conv)(conv_2)
    # 4
    conv_3 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(dropout_2)
    dropout_3 = Dropout(p_conv)(conv_3)
    maxpool_3 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_3)
    # 6
    conv_4 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(maxpool_3)
    dropout_4 = Dropout(p_conv)(conv_4)
    # 7
    conv_5 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(dropout_4)
    dropout_5 = Dropout(p_conv)(conv_5)
    maxpool_5 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_5)
    # 9 
    conv_6 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(maxpool_5)
    dropout_6 = Dropout(p_conv)(conv_6)
    # 10    
    conv_7 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(dropout_6)
    dropout_7 = Dropout(p_conv)(conv_7)
    # 11
    conv_8 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(dropout_7)
    dropout_8 = Dropout(p_conv)(conv_8)
    # 12
    conv_9 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                    activation= keras.layers.LeakyReLU(alpha=0.5),
                   )(dropout_8)
    dropout_9 = Dropout(p_conv)(conv_9)
    maxpool_9 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_9)
    # 14
    conv_10 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                     activation= keras.layers.LeakyReLU(alpha=0.5),
                    )(maxpool_9)
    dropout_10 = Dropout(p_conv)(conv_10)
    # 15
    conv_11 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                     activation= keras.layers.LeakyReLU(alpha=0.5),
                    )(dropout_10)
    dropout_11 = Dropout(p_conv)(conv_11)
    # 16
    conv_12 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                     activation= keras.layers.LeakyReLU(alpha=0.5),
                    )(dropout_11)
    dropout_12 = Dropout(p_conv)(conv_12)
    # 17
    conv_13 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                     activation= keras.layers.LeakyReLU(alpha=0.5),
                    )(dropout_12)
    dropout_13 = Dropout(p_conv)(conv_13)
    maxpool_13 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_13)
    # 19, special dropout between phases with p=1/2 
    dropout_inter = Dropout(0.5)(maxpool_13)
    # 20 Dense phase 
    # Begins with Maxout layer, which 
    maxout_1 = Dense(units=1024, 
                     activation=None,)(dropout_inter)
    # need to wrap operation in Lambda to count as a layer
    maxout_2 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=3))(maxout_1)
    
    # 22 Concatenate with processed img, take both eyes into account
    img_dim_input = Input(shape=(2,), batch_size=batch_size, name="imgdim")
    concat = concatenate([maxout_2, img_dim_input], axis=1)
    
    # 24
    # flatten = Reshape((-1, concat.output_shape()[1] * 2))(concat)
    print(concat.shape)
    # use lambda for custom reshape that's capable of changing batch_size as well
    flatten = Lambda(lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2)))(concat)
    dense_droupout_0 = Dropout(0.5)(flatten)
    # 26
    dense_1 = Dense(units=1024,
                    activation=None,
                   )(dense_droupout_0)
    dense_maxpool_1 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(dense_1)
    dense_dropout_1 = Dropout(0.5)(dense_maxpool_1)
    
    # 29
    dense_2 = Dense(units=n_classes*2,
                    activation=None,
                   )(dense_dropout_1)
    softmax_flatten = Lambda(lambda x: tf.reshape(x, (batch_size, n_classes)))(dense_2)
    softmax = keras.layers.Softmax()(softmax_flatten)
    
    model = Model(inputs=[main_input, img_dim_input], outputs=[softmax])
    
    return model

In [32]:
model = build_model()
model.summary()

(7, 7, 512, 2)
(64, 514)
(512, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (64, 512, 512, 3)    0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (64, 256, 256, 32)   4736        input_5[0][0]                    
__________________________________________________________________________________________________
dropout_60 (Dropout)            (64, 256, 256, 32)   0           conv2d_52[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_20 (MaxPooling2D) (64, 127, 127, 32)   0           dropout_60[0][0]                 
___________________________________________________________________________

In [39]:
model.layers[1].get_weights()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
x=concatenate([tf.zeros((1, 1)), tf.zeros((1, 1))])
x.get_shape()

In [ ]:
help(tf.TensorShape)

# Load model from pickle

build_model().summary()